In [1]:
import json
from datasets import Dataset
import re,os
import numpy as np
import pandas as pd
from utils import *
import random

In [2]:
df_mm_all = pd.read_csv('../../inference/results/llama/results-all-mm.csv')
# df_mm_cot = pd.read_csv('../../inference/results/llama/llama-mm-cot.csv')
# df_mm_sub = pd.read_csv('../../inference/results/llama/llama-mm-sub.csv')
df_gsm_test = pd.read_csv('../../inference/results/llama/results-all-test.csv')
df_gsm_train = pd.read_csv('../../inference/results/llama/results-all-train.csv')
df_mm_all['question'] = df_mm_all['question'].apply(lambda x: x[:-1])
df_gsm_test['question'] = df_gsm_test['question'].apply(lambda x: x[:-1])
df_gsm_train['question'] = df_gsm_train['question'].apply(lambda x: x[:-1])

In [3]:
dfs = np.array_split(df_mm_all, 20)
example_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the following Math Word Problem by firstly generating a strategy, then solving the problem according to the generated strategy.

Input:
Question: {}
<incorrect>None

Response:
<correct>"""

for i in range(len(dfs)):
    temp = {'text': [example_prompt.format(x) for x in dfs[i]['question']],
            'correct-answer':list(dfs[i]['correct-answer']),
            'question': [x for x in dfs[i]['question']]}
    with open('/cluster/project/sachan/liuron/Thesis/experiments/rl/epoch2/data/parts/part{}.json'.format(i+1), 'w') as json_file:
        json.dump(temp,json_file)

/cluster/home/liuron/.local/lib/python3.11/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


**Run the code below after running sample.py**

In [2]:
train = []
val = []
directory_path = '/cluster/project/sachan/liuron/Thesis/experiments/rl/epoch2/data/train/gemma'
for path in os.listdir(directory_path):
    with open(directory_path + '/' + path, 'r') as json_file:
        if 'train' in path:
            train.extend(json.load(json_file)['text'])
        else:
            val.extend(json.load(json_file)['text'])
with open('/cluster/project/sachan/liuron/Thesis/experiments/rl/epoch2/data/train.json', 'w') as json_file:
        json.dump({'text':train},json_file)
with open('/cluster/project/sachan/liuron/Thesis/experiments/rl/epoch2/data/val.json', 'w') as json_file:
        json.dump({'text':val},json_file)

In [3]:
with open('/cluster/project/sachan/liuron/Thesis/experiments/rl/epoch2/data/train.json', 'r') as json_file:
        dataset_train = json.load(json_file)['text']
with open('/cluster/project/sachan/liuron/Thesis/experiments/rl/epoch2/data/val.json', 'r') as json_file:
        dataset_val = json.load(json_file)['text']
dataset = dataset_train + dataset_val
print(len(dataset_train))
print(len(dataset_val))

194480
21617


In [4]:
def filter_incorrect_tag(dataset):
    res = []
    for i in range(len(dataset)):
        response_part = dataset[i].split('\nResponse:\n')[1]
        prompt_part = '<incorrect>' + dataset[i].split('\nResponse:\n')[0]
        if '<correct>Sub-questioning' in response_part and '?' not in response_part:
            continue
        if '<incorrect>Sub-questioning' in prompt_part and '?' not in prompt_part.split('<incorrect>Sub-questioning')[1]:
            continue
        res.append(dataset[i])
    return res

dataset_train = filter_incorrect_tag(dataset_train)
dataset_val = filter_incorrect_tag(dataset_val)
print(len(dataset_train))
print(len(dataset_val))
random.shuffle(dataset_train)
random.shuffle(dataset_val)
with open('/cluster/project/sachan/liuron/Thesis/experiments/rl/epoch2/data/train.json', 'w') as json_file:
    json.dump({'text':dataset_train},json_file)
with open('/cluster/project/sachan/liuron/Thesis/experiments/rl/epoch2/data/val.json', 'w') as json_file:
    json.dump({'text':dataset_val},json_file)

194480
21617
111113
21512
